In [1]:
import pandas as pd

In [2]:
import os,uuid

In [3]:
reprocess_datas_directory = f"{os.getcwd()}/../datas_to_english"

if not os.path.isdir(reprocess_datas_directory):
    os.mkdir(reprocess_datas_directory)

In [4]:
print(*list(filter(lambda x: x.endswith('.xlsx'),os.listdir())),sep="\n")

stadium_information.xlsx
kleague_player_information.xlsx
kleague_player_league_history.xlsx
previous_league_history.xlsx
team_information.xlsx


## Install dependencies

```bash
!conda install openpyxl -y
!conda install pandas -y
```

## kleague_player_information

In [5]:
df = pd.read_excel('kleague_player_information.xlsx',index_col=0)

### Preprocess country

In [6]:
country = set(df['국적'])
country

{'브라질', '한국', '호주'}

In [7]:
country_mapping = {
    '브라질' : 'Brazil',
    '한국' : 'Korea',
    '호주' : 'Australia'
}
df['국적'] = pd.Series(list(map(lambda x: country_mapping[x],df['국적'])))

### Preprocess team

In [8]:
team = set(df['소속구단'])
team_map_str = { i : "" for i in team}
team_map_str

{'대구': '',
 '김포': '',
 '제주': '',
 '경남': '',
 '전북': '',
 '서울E': '',
 '포항': '',
 '인천': '',
 '수원': '',
 '김천': '',
 '부천': '',
 '대전': '',
 '전남': '',
 '울산': '',
 '광주': '',
 '안양': '',
 '성남': '',
 '충남아산': '',
 '수원FC': '',
 '강원': '',
 '부산': '',
 '안산': '',
 '서울': ''}

In [9]:
# Team name mapping
team_mapping = {
     '울산': 'Ulsan',
     '전남': 'Jeonnam',
     '안산': 'Ansan',
     '서울': 'Seoul',
     '경남': 'Kyeongnam',
     '전북': 'Jeonbook',
     '대전': 'Daejeon',
     '인천': 'Incheon',
     '수원': 'Suwon',
     '충남아산': 'Choongnam_Asan',
     '광주': 'Gwangju',
     '서울E': 'Seoul_E',
     '김천': 'Kimcheon',
     '부산': 'Busan',
     '제주': 'Jeju',
     '대구': 'Daegu',
     '포항': 'Pohang',
     '안양': 'Anyang',
     '부천': 'Bucheon',
     '성남': 'Seongnam',
     '강원': 'Kangwon',
     '수원FC': 'Suwon_FC',
     '김포': 'Gimpo'
}

# Team id mapping
team_id_mapping = { i : uuid.uuid4() for i in team_mapping.values() }

df['소속구단'] = pd.Series(list(map(lambda x: team_mapping[x],df['소속구단'])))

In [10]:
import faker

faker = faker.Faker()

### Preprocess player's name and ID randomly again

In [11]:
player = df['이름']
random_name_set = list()

for _ in range(len(player)):
    n = faker.name()
    while True:
        if n not in random_name_set:
            random_name_set.append(n)
            break
        n = faker.name()
            
player_bucket = dict()
for i,j in enumerate(player):
    name = j.split('_')[0]
    if name in player_bucket.keys():
        name = f"{name}{len(list(filter(lambda x: x.startswith(name),player_bucket.keys()))) + 1}"
    player_bucket[name] = {
        'ran_name' : random_name_set[i],
        'id' : j.split('_')[1]
    }

In [12]:
player_bucket

{'갈레고': {'ran_name': 'Rebecca Bell',
  'id': '7edb1a28-8838-4051-b806-90e8804863d0'},
 '강민수': {'ran_name': 'Hannah Jones',
  'id': 'cf783b0e-ec16-4618-9a1b-28e914ff2ec3'},
 '강상윤': {'ran_name': 'Kristina Simmons',
  'id': '328087fe-0fc5-4e5c-a4bb-1fe9970b2907'},
 '강상희': {'ran_name': 'Brent Nguyen',
  'id': '58f2c7fa-2ad7-49a1-89c9-4782d2464e23'},
 '강성진': {'ran_name': 'Charles Rose',
  'id': 'b2567ff4-3495-4c5e-8f99-4a8d31166610'},
 '강영석': {'ran_name': 'Julie Mack',
  'id': 'dfcb8643-d8af-4b90-a527-b33a9326201b'},
 '강윤구': {'ran_name': 'Timothy Rodriguez',
  'id': '43ae7638-09bf-4e1b-9879-99824b70638c'},
 '강윤성': {'ran_name': 'Olivia Phillips',
  'id': '67fd61dd-e574-40fe-9dd4-d504984ef925'},
 '강의빈': {'ran_name': 'Dawn Davis',
  'id': 'bb5ea942-d23e-48f3-ad90-2346e11dcecc'},
 '강의찬': {'ran_name': 'Steven Kirk',
  'id': '0e389c79-436f-49bc-bedc-806d87338e29'},
 '강재우': {'ran_name': 'Tyler Miller',
  'id': '5662db9c-17bd-4e00-9b3d-e5090b7c3b54'},
 '강준모': {'ran_name': 'Travis Sanford',
  'id': 

In [13]:
col_raname = pd.Series(list(map(lambda x: x['ran_name'],player_bucket.values())),name="raname")

In [14]:
col_id = pd.Series(list(map(lambda x: x['id'],player_bucket.values())),name="id")

In [15]:
df_team_id = pd.Series(list(map(lambda x: team_id_mapping[x],df['소속구단'])),name="teamid")

In [16]:
df = pd.concat([df,col_raname,col_id,df_team_id],axis=1)

In [17]:
df

,이름,영문명,소속구단,포지션,배번,국적,키,몸무게,생년월일,league_type,player_image,raname,id,teamid
0,갈레고_7edb1a28-8838-4051-b806-90e8804863d0,ISIDIO JEFFERSON FERNANDO,Kangwon,FW,11,Brazil,177,71,1997/04/04,kleague1,-,Rebecca Bell,7edb1a28-8838-4051-b806-90e8804863d0,7508a993-6eee-47d8-a537-5d5aec6bb902
1,강민수_cf783b0e-ec16-4618-9a1b-28e914ff2ec3,MINSOO KANG,Incheon,DF,4,Korea,186,76,1986/02/14,kleague1,https://kleague-admin-test.s3.ap-northeast-2.a...,Hannah Jones,cf783b0e-ec16-4618-9a1b-28e914ff2ec3,d88ef14f-d2dc-4721-aacc-1f632f7347eb
2,강상윤_328087fe-0fc5-4e5c-a4bb-1fe9970b2907,KANG Sangyoon,Jeonbook,MF,36,Korea,171,64,2004/05/31,kleague1,-,Kristina Simmons,328087fe-0fc5-4e5c-a4bb-1fe9970b2907,c67f321d-fab0-4e2a-bdaf-5801469d37b5
3,강상희_58f2c7fa-2ad7-49a1-89c9-4782d2464e23,Sanghee KANG,Seoul,DF,28,Korea,178,70,1998/03/07,kleague1,https://kleague-admin-test.s3.ap-northeast-2.a...,Brent Nguyen,58f2c7fa-2ad7-49a1-89c9-4782d2464e23,05d44392-c3eb-4e0e-a3df-28a547d2b0d3
4,강성진_b2567ff4-3495-4c5e-8f99-4a8d31166610,Seongjin KANG,Seoul,FW,29,Korea,180,75,2003/03/26,kleague1,https://kleague-admin-test.s3.ap-northeast-2.a...,Charles Rose,b2567ff4-3495-4c5e-8f99-4a8d31166610,05d44392-c3eb-4e0e-a3df-28a547d2b0d3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,김인성_07cd55a8-5769-4379-af74-88d6b5012f20,Insung KIM,Seoul_E,FW,7,Korea,180,77,1989/09/09,kleague2,https://kleague-admin-test.s3.ap-northeast-2.a...,Michelle Haynes,07cd55a8-5769-4379-af74-88d6b5012f20,b623377a-c762-48a0-9124-9bb8a345dae6
156,김재봉_a47ad859-fc84-46a5-89d8-537945c6de99,Jaebong KIM,Gwangju,DF,34,Korea,188,78,1996/09/06,kleague2,https://kleague-admin-test.s3.ap-northeast-2.a...,Craig Padilla,a47ad859-fc84-46a5-89d8-537945c6de99,fe67fd6b-7f29-448c-a4ef-241e6f3e6b43
157,김재우_2d421aa3-7256-4ee5-8d4a-e033ef004349,Jaewoo KIM,Daejeon,DF,4,Korea,187,84,1998/02/06,kleague2,https://kleague-admin-test.s3.ap-northeast-2.a...,Angela White,2d421aa3-7256-4ee5-8d4a-e033ef004349,97764f5d-102d-44f7-b018-515714df51bf
158,김정민_47a39877-b81c-4841-8f17-0062194220c9,Jeongmin KIM,Ansan,DF,28,Korea,172,65,1995/09/06,kleague2,https://kleague-admin-test.s3.ap-northeast-2.a...,John Ortiz,47a39877-b81c-4841-8f17-0062194220c9,c9986db9-5430-4be4-8bbd-d816e112ac1c


In [18]:
df.to_excel(f'{reprocess_datas_directory}/kleague_player_information_renew.xlsx')

## kleague_player_league_history

In [19]:
kplh = pd.read_excel('kleague_player_league_history.xlsx',index_col=0)

In [20]:
kplh

,연도,리그,소속,출장,득점,도움,골킥,코너킥,오프사이드,슈팅,파울,실점,경고,퇴장,이름
0,2022,하나원큐 K리그1 2022,강원,14,3,0,0,0,0,12,11,0,2,0,갈레고_9d832ebf-c10e-4149-ab48-a0dd70525435
1,2022,하나원큐 K리그1 2022,인천,25,0,0,0,0,2,1,11,0,1,1,강민수_457d8718-1590-40ad-8b5b-972e86d25a61
2,2021,하나원큐 K리그1 2021,인천,17,0,0,0,0,0,0,11,0,1,0,강민수_457d8718-1590-40ad-8b5b-972e86d25a61
3,2020,하나원큐 K리그1 2020,부산,20,1,0,0,0,0,2,19,0,3,0,강민수_457d8718-1590-40ad-8b5b-972e86d25a61
4,2019,하나원큐 K리그1 2019,울산,23,3,0,0,0,1,7,20,0,7,0,강민수_457d8718-1590-40ad-8b5b-972e86d25a61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,2019,하나원큐 K리그2 2019,부천,25,1,3,0,0,2,7,29,0,2,0,김재우_521e6d28-454e-4ee2-9b5a-048489036f46
858,2018,KEB하나은행 K리그2 2018,부천,1,0,0,0,0,0,0,0,0,0,0,김재우_521e6d28-454e-4ee2-9b5a-048489036f46
859,2022,하나원큐 K리그2 2022,안산,0,0,0,0,0,0,0,0,0,0,0,김정민_926480b4-3ff7-4afb-9d04-f4e03fa6edbb
860,2022,하나원큐 K리그2 2022,부산,19,1,0,0,1,0,11,7,0,1,0,김정민_0f44a839-2782-4341-8b54-8868359d0a35


In [21]:
league = set(kplh['소속'])
# 현재 리그에 존재하는 팀들에 대한 히스토리만 필터링 
# -> K-league 홈페이지처럼 모든 데이터를 다 가지고 있는 상태가 아니므로 현재 있는 데이터에 대해서만 처리할 수 있도록 데이터를 가공한다.
n_kplh = kplh.copy().loc[kplh['소속'].isin(team_mapping.keys())]
# index reset for numbering
n_kplh.reset_index(drop=True, inplace=True)

In [22]:
# 소속 전처리
n_kplh['소속'] = pd.Series(list(map(lambda x:team_mapping[x],n_kplh['소속'])),name='소속')
# 이름 전처리
n_kplh['이름'] = pd.Series(list(map(lambda x:x.split('_')[0],n_kplh['이름'])),name='이름')
# 현재 플레이어 명단에만 존재하는 

In [23]:
n_kplh = n_kplh.loc[n_kplh['이름'].isin(player_bucket.keys())]

In [24]:
raname = pd.Series([ player_bucket[i]['ran_name'] for i in n_kplh['이름']],name="raname")
_id = pd.Series([player_bucket[i]['id'] for i in n_kplh['이름']],name="id")
kplh_teamid_col = pd.Series(list(map(lambda x: team_id_mapping[x],n_kplh['소속'])),name="teamid")
n_kplh = pd.concat([n_kplh,raname,_id,kplh_teamid_col],axis=1)

In [25]:
{ i : "" for i in set(n_kplh['리그'])}

{'하나원큐 K리그1 2020': '',
 '현대오일뱅크 K리그 챌린지': '',
 '현대오일뱅크 K리그 2011': '',
 '현대오일뱅크 K리그 2012': '',
 '하나원큐 K리그2 2019': '',
 '하나원큐 K리그1 2019': '',
 'KEB하나은행 K리그 챌린지 2017': '',
 '현대오일뱅크 K리그 클래식 2016': '',
 '삼성 하우젠 K-리그 2005': '',
 'KEB하나은행 K리그2 2018': '',
 'KEB하나은행 K리그1 2018': '',
 '2000 삼성디지털 K-리그': '',
 '98 현대컵 K-리그': '',
 '삼성 하우젠 K-리그 2008': '',
 '99 바이코리아컵 K-리그': '',
 '현대오일뱅크 K리그 챌린지 2016': '',
 'KEB하나은행 K리그 클래식 2017': '',
 '2009 K-리그': '',
 '현대오일뱅크 K리그 클래식 2014': '',
 '97 라피도컵 프로축구대회': '',
 '삼성 하우젠 K-리그 2003': '',
 '쏘나타 K리그 2010 ': '',
 '하나원큐 K리그1 2022': '',
 '현대오일뱅크 K리그 챌린지 2014': '',
 '삼성 하우젠 K-리그 2006': '',
 '하나원큐 K리그1 2021': '',
 '2001 포스코 K-리그': '',
 '하나원큐 K리그2 2021': '',
 '현대오일뱅크 K리그 클래식 2015': '',
 '삼성 하우젠 K-리그 2004': '',
 '현대오일뱅크 K리그 클래식': '',
 '현대오일뱅크 K리그 챌린지 2015': '',
 '2002 삼성 파브 K-리그': '',
 '하나원큐 K리그2 2022': '',
 '하나원큐 K리그2 2020': '',
 '삼성 하우젠 K-리그 2007': '',
 '96 라피도컵 프로축구대회': ''}

In [26]:
league_mapping = {
 '2009 K-리그': '2009 K-league',
 '현대오일뱅크 K리그 클래식 2016': 'Hyundai K-league Classic 2016',
 '하나원큐 K리그1 2022': 'Hanaone Q K-league1 2022',
 '하나원큐 K리그2 2019': 'Hanaone Q K-league2 2019',
 'KEB하나은행 K리그 챌린지 2017': 'KEB Hana K-league Challenge 2017',
 'KEB하나은행 K리그1 2018': 'KEB Hana K-league1 2018',
 '현대오일뱅크 K리그 2011': 'Hyundai K-league 2011',
 '하나원큐 K리그2 2021': 'Hanaone Q K-league2 2021',
 '현대오일뱅크 K리그 클래식 2014': 'Hyundai K-leageue Classic 2014',
 '하나원큐 K리그1 2020': 'Hanaone Q K-league1 2020',
 '현대오일뱅크 K리그 2012': 'Hyundai K-league 2012',
 '현대오일뱅크 K리그 챌린지 2016': 'Hyundai K-leageue Challenge 2016',
 '2001 포스코 K-리그': 'K-league POSCO 2001',
 '삼성 하우젠 K-리그 2006': 'Samsung K-league 2006',
 '삼성 하우젠 K-리그 2007': 'Samsung K-league 2007',
 '삼성 하우젠 K-리그 2005': 'Samsung K-league 2005',
 '현대오일뱅크 K리그 챌린지 2015': 'Hyundai K-leageue Challenge 2015',
 '현대오일뱅크 K리그 클래식': 'Hyundai K-leageue Classic',
 '현대오일뱅크 K리그 챌린지 2014': 'Hyundai K-leageue Challenge 2014',
 '2002 삼성 파브 K-리그': 'Samsung K-league 2002',
 '97 라피도컵 프로축구대회': '97 Rapidocup',
 '삼성 하우젠 K-리그 2008': 'Samsung K-league 2008',
 '쏘나타 K리그 2010 ': 'Sonata K-league 2010',
 '하나원큐 K리그1 2021': 'Hanaone Q K-league1 2021',
 '삼성 하우젠 K-리그 2004': 'Samsung K-league 2004',
 '2000 삼성디지털 K-리그': '2000 Samsung K-league',
 '96 라피도컵 프로축구대회': '96 Rapidocup',
 '98 현대컵 K-리그': '98 Hyundaicup K-league',
 '삼성 하우젠 K-리그 2003': 'Samsung K-league 2003',
 '하나원큐 K리그2 2022': 'Hanaone Q K-league2 2022',
 '하나원큐 K리그1 2019': 'Hanaone Q K-league1 2019',
 '현대오일뱅크 K리그 챌린지': 'Hyundai K-leageue Challenge',
 '하나원큐 K리그2 2020': 'Hanaone Q K-league2 2020',
 '현대오일뱅크 K리그 클래식 2015': 'Hyundai K-leageue Classic 2015',
 '99 바이코리아컵 K-리그': '99 By-Koreacup',
 'KEB하나은행 K리그2 2018': 'KEB Hana K-league2 2018',
 'KEB하나은행 K리그 클래식 2017': 'KEB Hana K-league Classic 2017'}

n_kplh['리그'] = pd.Series(list(map(lambda x: league_mapping[x],n_kplh['리그'])),name="리그")

In [27]:
n_kplh.to_excel(f'{reprocess_datas_directory}/kleague_player_league_history_renew.xlsx')

In [28]:
n_kplh

,연도,리그,소속,출장,득점,도움,골킥,코너킥,오프사이드,슈팅,파울,실점,경고,퇴장,이름,raname,id,teamid
0,2022,Hanaone Q K-league1 2022,Kangwon,14,3,0,0,0,0,12,11,0,2,0,갈레고,Rebecca Bell,7edb1a28-8838-4051-b806-90e8804863d0,7508a993-6eee-47d8-a537-5d5aec6bb902
1,2022,Hanaone Q K-league1 2022,Incheon,25,0,0,0,0,2,1,11,0,1,1,강민수,Hannah Jones,cf783b0e-ec16-4618-9a1b-28e914ff2ec3,d88ef14f-d2dc-4721-aacc-1f632f7347eb
2,2021,Hanaone Q K-league1 2021,Incheon,17,0,0,0,0,0,0,11,0,1,0,강민수,Hannah Jones,cf783b0e-ec16-4618-9a1b-28e914ff2ec3,d88ef14f-d2dc-4721-aacc-1f632f7347eb
3,2020,Hanaone Q K-league1 2020,Busan,20,1,0,0,0,0,2,19,0,3,0,강민수,Hannah Jones,cf783b0e-ec16-4618-9a1b-28e914ff2ec3,82f70bed-ebcd-4751-8245-7221838bc31d
4,2019,Hanaone Q K-league1 2019,Ulsan,23,3,0,0,0,1,7,20,0,7,0,강민수,Hannah Jones,cf783b0e-ec16-4618-9a1b-28e914ff2ec3,7c925ac6-1512-497a-8807-19aee53531d2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,2019,Hanaone Q K-league2 2019,Bucheon,25,1,3,0,0,2,7,29,0,2,0,김재우,Angela White,2d421aa3-7256-4ee5-8d4a-e033ef004349,b9af5cac-c771-40c1-8e03-8c46bd84538d
790,2018,KEB Hana K-league2 2018,Bucheon,1,0,0,0,0,0,0,0,0,0,0,김재우,Angela White,2d421aa3-7256-4ee5-8d4a-e033ef004349,b9af5cac-c771-40c1-8e03-8c46bd84538d
791,2022,Hanaone Q K-league2 2022,Ansan,0,0,0,0,0,0,0,0,0,0,0,김정민,John Ortiz,47a39877-b81c-4841-8f17-0062194220c9,c9986db9-5430-4be4-8bbd-d816e112ac1c
792,2022,Hanaone Q K-league2 2022,Busan,19,1,0,0,1,0,11,7,0,1,0,김정민,John Ortiz,47a39877-b81c-4841-8f17-0062194220c9,82f70bed-ebcd-4751-8245-7221838bc31d


## Previous league history

In [29]:
plh = pd.read_excel('previous_league_history.xlsx',index_col=0)

In [30]:
# Filter league that exists in player's previous league
n_plh = plh.copy().loc[plh['리그'].isin(league_mapping.keys())]
n_plh.reset_index(drop=True, inplace=True)

In [31]:
n_plh['리그'] = pd.Series(list(map(lambda x: league_mapping[x],n_plh['리그'])))

In [32]:
n_plh.to_excel(f'{reprocess_datas_directory}/previous_league_history_renew.xlsx')

## Stadium Information

In [33]:
sti = pd.read_excel('stadium_information.xlsx',index_col=0)

In [34]:
sti['클럽'] = pd.Series(list(map(lambda x: team_mapping[x],sti['클럽'])))
sti_club_id = pd.Series(list(map(lambda x: team_id_mapping[x],sti['클럽'])),name="teamid")
sti = pd.concat([sti,sti_club_id],axis=1)

In [35]:
{ i : "" for i in sti['경기장']}

{'울산문수축구경기장*': '',
 '전주월드컵경기장*': '',
 '포항스틸야드*': '',
 '인천축구전용경기장*': '',
 '제주월드컵경기장*': '',
 '강릉종합경기장': '',
 '수원월드컵경기장*': '',
 'DGB대구은행파크*': '',
 '서울월드컵경기장*': '',
 '수원종합운동장': '',
 '김천종합운동장': '',
 '탄천종합운동장': '',
 '광주축구전용구장': '',
 '대전월드컵경기장*': '',
 '안양종합운동장': '',
 '창원축구센터*': '',
 '부천종합운동장': '',
 'Stadium Unknown': '',
 '목동운동장': '',
 '김포솔터축구장*': '',
 '와~스타디움': '',
 '구덕운동장': '',
 '광양축구전용구장*': ''}

In [36]:
previous_league_mapping = {
 '울산문수축구경기장*': 'Ulsan Stadium',
 '전주월드컵경기장*': 'Jeonju worldcup Stadium',
 '포항스틸야드*': 'Pohang steal yard',
 '인천축구전용경기장*': 'Incheon Stadium',
 '제주월드컵경기장*': 'Jeju worldcup Stadium',
 '강릉종합경기장': 'Gangleung Stadium',
 '수원월드컵경기장*': 'Suwon worldcup Stadium',
 'DGB대구은행파크*': 'DGB Daegu Park',
 '서울월드컵경기장*': 'Seoul worldcup Stadium',
 '수원종합운동장': 'Suwon Stadium',
 '김천종합운동장': 'Kimcheon Stadium',
 '탄천종합운동장': 'Tancheon Stadium',
 '광주축구전용구장': 'Gwangju Stadium',
 '대전월드컵경기장*': 'Dajeon worldcup Stadium',
 '안양종합운동장': 'Anyang Stadium',
 '창원축구센터*': 'Changwon Stadium',
 '부천종합운동장': 'Bucheon Stadium',
 'Stadium Unknown': 'Choongnam Stadium',
 '목동운동장': 'Mokdong Stadium',
 '김포솔터축구장*': 'Gimpo Stadium',
 '와~스타디움': 'Wa~ Stadium',
 '구덕운동장': 'Goodeok Stadium',
 '광양축구전용구장*': 'Gwangyang Stadium'
}
sti['경기장'] = pd.Series(list(map(lambda x: previous_league_mapping[x],sti['경기장'])))

In [37]:
sti.to_excel(f'{reprocess_datas_directory}/stadium_information.xlsx')

## Team information

In [38]:
tei = pd.read_excel('team_information.xlsx',index_col=0)

In [39]:
# Filter club
tei['클럽'] = pd.Series(list(map(lambda x: team_mapping[x],tei['클럽'])))
tei_club_id = pd.Series(list(map(lambda x: team_id_mapping[x],tei['클럽'])),name="teamid")
tei = pd.concat([tei,tei_club_id],axis=1)

In [40]:
w_or_l = {
    '승' : 'W',
    '패' : 'L',
    '무' : 'D'
}

tei['최근 홈 5경기'] = pd.Series(list(map(lambda x: ' '.join(list(map(lambda y: w_or_l[y],x.split(' ')))), tei['최근 홈 5경기'])))

In [41]:
tei.to_excel(f'{reprocess_datas_directory}/team_information.xlsx')